# Assignment 2: Task 1

โดย

ตุลธร วงศ์ชัย รหัสนักศึกษา 63070224

ธนภัทร ธีรรัตตัญญู รหัสนักศึกษา 63070227

> **README**:
> - 'Run all' is not recommended. 
> - Section 1 must be run before any other section.
> - Read and follow instruction of each section carefully if it is provided.


# 1. Init Project

> **Instruction**: Set `ROOT_PATH` before running this section.

## 1.1 Import Libraries and Packages

In [60]:
import os
import time
import requests
from bs4 import BeautifulSoup
from pprint import pprint

import numpy as np
import pandas as pd

## 1.2 Set Root Path

* If you initialize this project for first time, run code cell R1
* Otherwise, set your own root path in code cell R2

In [61]:
# ### Code Cell R1 ###

# ROOT_PATH = os.getcwd() # Get current working directory

# # Create /data directory
# data_dir = os.path.join(ROOT_PATH, 'data')
# os.mkdir(data_dir) 

# # Create /data/raw directory
# data_target_dir = os.path.join(ROOT_PATH, 'data/raw')
# os.mkdir(data_target_dir)

# # Create /data/datastore directory
# data_datastore_dir = os.path.join(ROOT_PATH, 'data/datastore')
# os.mkdir(data_datastore_dir) 

# # Create /data/target directory
# data_target_dir = os.path.join(ROOT_PATH, 'data/target')
# os.mkdir(data_target_dir) 

In [62]:
### Code Cell R2 ###

# Set your own root path here
ROOT_PATH = os.getcwd()

# 2. Text Scraping from Web

## 2.1 กำหนดค่าคงที่และสร้างฟังก์ชันทีใช้งานบ่อย

2.1.1 กำหนดค่าคงที่ BASE_URL ตามเซลล์โค้ดที่ 2.1.1

In [63]:
### Code Cell 2.1.1 ###

BASE_URL = 'http://www.it.kmitl.ac.th/~teerapong/news_archive/'

2.1.2 สร้างฟังก์ชันสำหรับส่ง GET request ไปยัง URL ที่ต้องการ พร้อมตรวจสอบว่าการส่งนั้นสำเร็จหรือไม่ (มี status เป็น 200) ถ้าไม่สำเร็จ ให้ทำการ raise Exception ตามเซลล์โค้ดที่ 2.1.2

In [64]:
### Code Cell 2.1.2 ###

def get_request(url:str,**kwargs)->requests.models.Response:
  """
  Call GET request using requests package 
  then return response if it is successful (status 200)
  otherwise throw an exception.

  Parameters
  ----------
  url : str
    URL to request
  **kwargs
    Other parameters for requests.get

  Returns
  ----------
  requests.models.Response
    Response of the request
  """
  res = requests.get(url, **kwargs)

  # If the request is not successful, throw an exception.  
  if (res.status_code != 200):
    print(res.text)
    err = Exception('The request is not successful. ' + 
                    f'Its status code is {res.status_code}. ' + 
                    f'The URL of request is {res.url}')
    raise err
  else:
    return res

## 2.2 เก็บ URL ของหน้ารวมบทความข่าวแต่ละเดือน

2.2.1 request หน้า index ของ News Article Archive และแปลงด้วย BeautifulSoup ตามเซลล์โค้ดที่ 2.2.1

In [65]:
### Code Cell 2.2.1 ###

index_url = BASE_URL + 'index.html'
res = get_request(index_url)
index_page = BeautifulSoup(res.content, 'lxml') # Parse the response
index_page

<!DOCTYPE html>
<html lang="en">
<head>
<title>Online News Archive</title>
<meta charset="utf-8"/>
<meta content="width=device-width, initial-scale=1" name="viewport"/>
<meta content="noindex" name="robots"/>
<meta content="news,articles,news" name="keywords"/>
<meta content="Breaking News | International Headlines" property="og:title"/>
<meta content="News Archive" property="og:site_name"/>
<meta content="Latest news and more from the definitive brand of quality news." property="og:description"/>
<link href="css/bootstrap.min.css" rel="stylesheet"/>
<script src="js/jquery-3.2.1.slim.min.js"></script>
<script src="js/popper.min.js"></script>
<script src="js/tether.min.js"></script>
<script src="js/jquery-3.2.1.slim.min.js"></script>
<script src="js/popper.min.js"></script>
<script src="js/tether.min.js"></script>
<script src="js/bootstrap.min.js"></script>
<style>
	  		.main{ padding: 0; text-align: center;}
	  		.footer{ padding: 6px;text-align: center; margin-top: 1em; }

	  		h1
	  

2.2.2 เนื่องจากในหน้า index จะมีรายการของเดือนที่มีทั้งหมดอยู่ใน `<li>` ซึ่งเชื่อมไปยังหน้ารวมบทความข่าวของเดือนนั้น ๆ จึงทำการดึง `<li>` tag ตามเซลล์โค้ดที่ 2.2.2.1 จากนั้นทำการดึงค่าของ `href` ของ `<a>` ที่อยู่ในแต่ละ `<li>` tag ที่ได้มา ตามเซลล์โค้ดที่ 2.2.2.2

In [66]:
### Cell Code 2.2.2.1  ###

tags = index_page.select('li')
tags

[<li>Articles — <a href="month-jan-2017.html">January</a> [118]</li>,
 <li>Articles — <a href="month-feb-2017.html">February</a> [124]</li>,
 <li>Articles — <a href="month-mar-2017.html">March</a> [116]</li>,
 <li>Articles — <a href="month-apr-2017.html">April</a> [118]</li>,
 <li>Articles — <a href="month-may-2017.html">May</a> [115]</li>,
 <li>Articles — <a href="month-jun-2017.html">June</a> [115]</li>,
 <li>Articles — <a href="month-jul-2017.html">July</a> [122]</li>,
 <li>Articles — <a href="month-aug-2017.html">August</a> [116]</li>,
 <li>Articles — <a href="month-sep-2017.html">September</a> [113]</li>,
 <li>Articles — <a href="month-oct-2017.html">October</a> [124]</li>,
 <li>Articles — <a href="month-nov-2017.html">November</a> [122]</li>,
 <li>Articles — <a href="month-dec-2017.html">December</a> [115]</li>]

In [67]:
### Cell Code 2.2.2.2  ###

month_links = list()
for tag in tags:
  link = tag.find('a')['href']
  month_links.append(link)
  print(link)

month-jan-2017.html
month-feb-2017.html
month-mar-2017.html
month-apr-2017.html
month-may-2017.html
month-jun-2017.html
month-jul-2017.html
month-aug-2017.html
month-sep-2017.html
month-oct-2017.html
month-nov-2017.html
month-dec-2017.html


## 2.3 เก็บข้อมูลหมวดหมู่และ URL ของบทความข่าวในแต่ละเดือน

In [68]:
def scrape_month_page(month_url):
  """
  Scrape title, category and link of available news articles from month page.

  Parameters
  ----------
  month_url : str
    URL of month page

  Returns
  ----------
  DataFrame
    DataFrame containing title, category and link of news articles
  """

  res = get_request(month_url)
  month_page = BeautifulSoup(res.content, 'lxml') # Parse the response
  
  articles = month_page.select('tbody>tr')

  article_titles = list()
  article_categories = list()
  article_links = list()

  for article in articles:
    category = article.select_one('td.category').string.strip()
    if (category!='N/A'):
      title = article.select_one('td.title>a').string
      link = article.select_one('td.title>a')['href']
      article_titles.append(title)
      article_categories.append(category)
      article_links.append(link)
  
  articles_df = pd.DataFrame({'title':article_titles,
                              'category':article_categories,
                              'link':article_links})  
  return articles_df

In [69]:
month_abbr_to_number = {
  'jan':'01',
  'feb':'02',
  'mar':'03',
  'apr':'04',
  'may':'05',
  'jun':'06',
  'jul':'07',
  'aug':'08',
  'sep':'09',
  'oct':'10',
  'nov':'11',
  'dec':'12'
}

for month_link in month_links:
  
  month_url = BASE_URL + month_link
  print(f'Scraping {month_url}')
  articles_df = scrape_month_page(month_url)
  print(f'Scraped {articles_df.shape[0]} article links')

  month = month_link[6:9] # Get month abbriviation from month_link
  month = month_abbr_to_number[month] # Convert to month number
  year = month_link[10:14] # Get year from month_link
  file_name = f'article_links_{year}_{month}.csv'
  file_path = f'{ROOT_PATH}/data/raw/{file_name}'
  articles_df.to_csv(file_path, index=False)
  print(f'Saved to ROOT_PATH/data/raw/{file_name}')
  print('-'*10)


Scraping http://www.it.kmitl.ac.th/~teerapong/news_archive/month-jan-2017.html
Scraped 118 article links
Saved to ROOT_PATH/data/raw/article_links_2017_01.csv
----------
Scraping http://www.it.kmitl.ac.th/~teerapong/news_archive/month-feb-2017.html
Scraped 122 article links
Saved to ROOT_PATH/data/raw/article_links_2017_02.csv
----------
Scraping http://www.it.kmitl.ac.th/~teerapong/news_archive/month-mar-2017.html
Scraped 116 article links
Saved to ROOT_PATH/data/raw/article_links_2017_03.csv
----------
Scraping http://www.it.kmitl.ac.th/~teerapong/news_archive/month-apr-2017.html
Scraped 117 article links
Saved to ROOT_PATH/data/raw/article_links_2017_04.csv
----------
Scraping http://www.it.kmitl.ac.th/~teerapong/news_archive/month-may-2017.html
Scraped 114 article links
Saved to ROOT_PATH/data/raw/article_links_2017_05.csv
----------
Scraping http://www.it.kmitl.ac.th/~teerapong/news_archive/month-jun-2017.html
Scraped 114 article links
Saved to ROOT_PATH/data/raw/article_links_201

## 2.4 เก็บข้อมูลจากหน้าบทความข่าว

In [70]:
article_links_month_files = os.listdir(f'{ROOT_PATH}/data/raw')
article_links_month_files

['article_links_2017_01.csv',
 'article_links_2017_02.csv',
 'article_links_2017_03.csv',
 'article_links_2017_04.csv',
 'article_links_2017_05.csv',
 'article_links_2017_06.csv',
 'article_links_2017_07.csv',
 'article_links_2017_08.csv',
 'article_links_2017_09.csv',
 'article_links_2017_10.csv',
 'article_links_2017_11.csv',
 'article_links_2017_12.csv']

In [72]:
def scrape_article_page(article_url):
  """
  Scrape article content from news article page with a given URL.

  Parameters
  ----------
  article_url : str
    URL of article page

  Returns
  ----------
  str
    Article content
  """

  res = get_request(article_url)
  article_page = BeautifulSoup(res.content, 'lxml') # Parse the response


  # Get content of article, which is in p tags without 'notice' class 
  # in div.main
  p_tags = article_page.select('div.main>p:not(.notice)')

  # Every article repeats its title in content section.
  # Due to inconsistent formating style, some articles repeat in b tag
  # while some repeat in p tag. In the later case, remove first item of
  # selected p_tags.
  b_tags = article_page.select('div.main>b')
  # If there is no b tag in content section, remove first item of
  # selected p_tags
  if (len(b_tags)==0):
    p_tags.pop(0)
  
  article_content = ''
  # Concatenate all paragraphs
  for p in p_tags:
    article_content = article_content + ' ' + p.text

  # Replace double space with single space and 
  # remove leading and trailing whitespace
  article_content = article_content.replace('  ',' ').strip() 

  return article_content

In [86]:
# Extract all contents to datestore folder
for article_links_month in article_links_month_files:
  article_df = pd.read_csv(f'{ROOT_PATH}/data/raw/{article_links_month}')
  article_contents = list()
  for _, article in article_df.iterrows():
    article_link = article['link']
    article_url = BASE_URL + article_link
    article_content = scrape_article_page(article_url)
    article_contents.append(article_content)

  article_contents_df = pd.DataFrame({'content':article_contents})
  article_contents_df.to_csv(f'{ROOT_PATH}/data/datastore/article_contents_{article_links_month[14:21]}.txt',sep='\n',
    index=False,header=False)



In [85]:
# Extract all category to target folder
for article_links_month in article_links_month_files:
    df = pd.read_csv(f'{ROOT_PATH}/data/raw/{article_links_month}')
    df['category'].to_csv(f'{ROOT_PATH}/data/target/article_targets_{article_links_month[14:21]}.txt',sep='\n',
  index=False,header=False)

In [102]:
df = pd.read_csv(f'{ROOT_PATH}/data/datastore/article_contents_2017_01.txt',header=None, delimiter="\r\n", names=['content'])
df

C:\Users\tunza\AppData\Local\Temp\ipykernel_11396\2088313385.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df = pd.read_csv(f'{ROOT_PATH}/data/datastore/article_contents_2017_01.txt',header=None, delimiter="\r\n", names=['content'])


,content
0,The sporting industry has come a long way sinc...
1,"""Shares in Europe's leading reinsurers and tra..."
2,"""BT is offering customers free internet teleph..."
3,"""Shares in UK banking group Barclays have rise..."
4,"""England centre Olly Barkley has been passed f..."
...,...
113,"""Former Worldcom boss Bernie Ebbers, who is ac..."
114,"""Details of the next generation of Microsoft's..."
115,"""Microsoft has said it will replace more than ..."
116,"""Yangtze Electric Power, the operator of China..."


# 3. Multi-Class Classification